In [1]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import numpy as np

In [2]:
#Import libraries and dependencies

import numpy as np
import pandas as pd
from pathlib import Path

Read CSV into pandas dataframe

In [3]:
# Load env data
load_dotenv()


True

In [4]:
# Assign Alpaca keys to variables
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")


In [5]:
# Initialize Alpaca connection
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version='v2'
)


In [6]:
# Set start and end dates for
start_date = pd.Timestamp('2020-01-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2020-12-31', tz='America/New_York').isoformat()


In [7]:
# Set file path:

#filepath = Path("../Resources/JPM.csv")

In [8]:
# Assign default ticker symbol
ticker_string = 'JPM'
ticker = [ticker_string]


In [9]:
# Assign default time frame
timeframe = '15Min'

In [10]:
# Get ticker historical data
df_ohlc_original = alpaca.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=end_date
).df


In [11]:
# Copy of original data
df_ohlc = df_ohlc_original

# Verify data in dataframe
df_ohlc = df_ohlc[ticker_string]
df_ohlc.head()


,open,high,low,close,volume
time,,,,,
2020-01-02 09:30:00-05:00,140.070,140.14,139.52,139.670,29953
2020-01-02 09:45:00-05:00,139.675,139.91,139.44,139.670,24343
2020-01-02 10:00:00-05:00,139.635,139.81,139.52,139.640,19021
2020-01-02 10:15:00-05:00,139.600,139.60,139.34,139.400,18856
2020-01-02 10:30:00-05:00,139.460,139.54,139.28,139.535,19695


In [12]:
#jpm_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)

#jpm_df.head()

In [13]:
#jpm_df.rename(columns={"Unnamed: 0": 'date'}, inplace=True)

In [14]:
#jpm_df.set_index("date",inplace =True)

In [15]:
#jpm_df.head()

In [16]:
"""
p_length = 20

def truehigh(h,yh):
    if yh >= h :
        truehigh_val= yh
    else:
        truehigh_val = h
    
    return truehigh_val



#df_ohlc['trueHigh'] = np.where(((df_ohlc['high'] > df_ohlc['truehigh'].shift(1))),df_ohlc['high'],df_ohlc['truehigh'])
"""


"\np_length = 20\n\ndef truehigh(h,yh):\n    if yh >= h :\n        truehigh_val= yh\n    else:\n        truehigh_val = h\n    \n    return truehigh_val\n\n\n\n#df_ohlc['trueHigh'] = np.where(((df_ohlc['high'] > df_ohlc['truehigh'].shift(1))),df_ohlc['high'],df_ohlc['truehigh'])\n"

In [17]:
df_ohlc.head()

,open,high,low,close,volume
time,,,,,
2020-01-02 09:30:00-05:00,140.070,140.14,139.52,139.670,29953
2020-01-02 09:45:00-05:00,139.675,139.91,139.44,139.670,24343
2020-01-02 10:00:00-05:00,139.635,139.81,139.52,139.640,19021
2020-01-02 10:15:00-05:00,139.600,139.60,139.34,139.400,18856
2020-01-02 10:30:00-05:00,139.460,139.54,139.28,139.535,19695


In [18]:
from stockstats import StockDataFrame as sdf

stockstats_df = sdf.retype(df_ohlc)

In [19]:
class StockDataFrame(pd.DataFrame):
    OPERATORS = ['le', 'ge', 'lt', 'gt', 'eq', 'ne']

    # Start of options.
    KDJ_PARAM = (2.0 / 3.0, 1.0 / 3.0)
    KDJ_WINDOW = 9

    BOLL_PERIOD = 20
    BOLL_STD_TIMES = 2

    MACD_EMA_SHORT = 12
    MACD_EMA_LONG = 26
    MACD_EMA_SIGNAL = 9

In [20]:
@classmethod
def _get_macd(cls, df):
    """ Moving Average Convergence Divergence
    This function will initialize all following columns.
    MACD Line (macd): (12-day EMA - 26-day EMA)
    Signal Line (macds): 9-day EMA of MACD Line
    MACD Histogram (macdh): MACD Line - Signal Line
    :param df: data
    :return: None
    """
    ema_short = 'close_{}_ema'.format(cls.MACD_EMA_SHORT)
    ema_long = 'close_{}_ema'.format(cls.MACD_EMA_LONG)
    ema_signal = 'macd_{}_ema'.format(cls.MACD_EMA_SIGNAL)
    fast = df[ema_short]
    slow = df[ema_long]
    df['macd'] = fast - slow
    df['macds'] = df[ema_signal]
    df['macdh'] = (df['macd'] - df['macds'])
    log.critical("NOTE: Behavior of MACDH calculation has changed as of "
                     "July 2017 - it is now 1/2 of previous calculated values")
    cls._drop_columns(df, [ema_short, ema_long, ema_signal])

In [21]:
#_get_macd(df_ohlc)

In [22]:


stock = sdf.retype(df_ohlc)
print(stock['macds'])

print(stock)

time
2020-01-02 09:30:00-05:00    0.000000
2020-01-02 09:45:00-05:00    0.000000
2020-01-02 10:00:00-05:00   -0.000381
2020-01-02 10:15:00-05:00   -0.003638
2020-01-02 10:30:00-05:00   -0.005334
                               ...   
2020-12-30 14:45:00-05:00    0.000626
2020-12-30 15:00:00-05:00   -0.006236
2020-12-30 15:15:00-05:00   -0.011130
2020-12-30 15:30:00-05:00   -0.011477
2020-12-30 15:45:00-05:00   -0.012439
Name: macds, Length: 6773, dtype: float64
                              open    high     low    close  volume      macd  \
time                                                                            
2020-01-02 09:30:00-05:00  140.070  140.14  139.52  139.670   29953  0.000000   
2020-01-02 09:45:00-05:00  139.675  139.91  139.44  139.670   24343  0.000000   
2020-01-02 10:00:00-05:00  139.635  139.81  139.52  139.640   19021 -0.000930   
2020-01-02 10:15:00-05:00  139.600  139.60  139.34  139.400   18856 -0.009996   
2020-01-02 10:30:00-05:00  139.460  139.54  139.2

In [23]:
print(stock['atr'])
print(stock)

time
2020-01-02 09:30:00-05:00         NaN
2020-01-02 09:45:00-05:00    0.470000
2020-01-02 10:00:00-05:00    0.376667
2020-01-02 10:15:00-05:00    0.349196
2020-01-02 10:30:00-05:00    0.324360
                               ...   
2020-12-30 14:45:00-05:00    0.219264
2020-12-30 15:00:00-05:00    0.211460
2020-12-30 15:15:00-05:00    0.203498
2020-12-30 15:30:00-05:00    0.211106
2020-12-30 15:45:00-05:00    0.228884
Name: atr, Length: 6773, dtype: float64
                              open    high     low    close  volume      macd  \
time                                                                            
2020-01-02 09:30:00-05:00  140.070  140.14  139.52  139.670   29953  0.000000   
2020-01-02 09:45:00-05:00  139.675  139.91  139.44  139.670   24343  0.000000   
2020-01-02 10:00:00-05:00  139.635  139.81  139.52  139.640   19021 -0.000930   
2020-01-02 10:15:00-05:00  139.600  139.60  139.34  139.400   18856 -0.009996   
2020-01-02 10:30:00-05:00  139.460  139.54  139.28 

In [24]:
print(stock['atr'])

time
2020-01-02 09:30:00-05:00         NaN
2020-01-02 09:45:00-05:00    0.470000
2020-01-02 10:00:00-05:00    0.376667
2020-01-02 10:15:00-05:00    0.349196
2020-01-02 10:30:00-05:00    0.324360
                               ...   
2020-12-30 14:45:00-05:00    0.219264
2020-12-30 15:00:00-05:00    0.211460
2020-12-30 15:15:00-05:00    0.203498
2020-12-30 15:30:00-05:00    0.211106
2020-12-30 15:45:00-05:00    0.228884
Name: atr, Length: 6773, dtype: float64


In [25]:
stock['atr']

time
2020-01-02 09:30:00-05:00         NaN
2020-01-02 09:45:00-05:00    0.470000
2020-01-02 10:00:00-05:00    0.376667
2020-01-02 10:15:00-05:00    0.349196
2020-01-02 10:30:00-05:00    0.324360
                               ...   
2020-12-30 14:45:00-05:00    0.219264
2020-12-30 15:00:00-05:00    0.211460
2020-12-30 15:15:00-05:00    0.203498
2020-12-30 15:30:00-05:00    0.211106
2020-12-30 15:45:00-05:00    0.228884
Name: atr, Length: 6773, dtype: float64

In [26]:
atr_column = pd.DataFrame(stock['atr'])

In [27]:
atr_column.head()

,atr
time,
2020-01-02 09:30:00-05:00,NaN
2020-01-02 09:45:00-05:00,0.470000
2020-01-02 10:00:00-05:00,0.376667
2020-01-02 10:15:00-05:00,0.349196
2020-01-02 10:30:00-05:00,0.324360


In [28]:
df_final = df_ohlc.join(atr_column)

In [29]:
df_final.head()

,open,high,low,close,volume,close_12_ema,close_26_ema,macd,macd_9_ema,macds,macdh,atr
time,,,,,,,,,,,,
2020-01-02 09:30:00-05:00,140.070,140.14,139.52,139.670,29953,139.670000,139.670000,0.000000,0.000000,0.000000,0.000000,NaN
2020-01-02 09:45:00-05:00,139.675,139.91,139.44,139.670,24343,139.670000,139.670000,0.000000,0.000000,0.000000,0.000000,0.470000
2020-01-02 10:00:00-05:00,139.635,139.81,139.52,139.640,19021,139.658291,139.659221,-0.000930,-0.000381,-0.000381,-0.000549,0.376667
2020-01-02 10:15:00-05:00,139.600,139.60,139.34,139.400,18856,139.576759,139.586754,-0.009996,-0.003638,-0.003638,-0.006358,0.349196
2020-01-02 10:30:00-05:00,139.460,139.54,139.28,139.535,19695,139.565413,139.574752,-0.009339,-0.005334,-0.005334,-0.004005,0.324360


In [30]:
p_length = 20
df_final['kcmid'] = df_final['close'].rolling(window=p_length).mean()
df_final['kcup'] = df_final['kcmid']+df_final['atr']
df_final['kclo'] = df_final['kcmid']-df_final['atr']
df_final.tail()

,open,high,low,close,volume,close_12_ema,close_26_ema,macd,macd_9_ema,macds,macdh,atr,kcmid,kcup,kclo
time,,,,,,,,,,,,,,,
2020-12-30 14:45:00-05:00,125.285,125.46,125.26,125.29,7778,125.361912,125.393819,-0.031907,0.000626,0.000626,-0.032533,0.219264,125.4420,125.661264,125.222736
2020-12-30 15:00:00-05:00,125.325,125.36,125.25,125.31,5098,125.353925,125.387610,-0.033685,-0.006236,-0.006236,-0.027448,0.211460,125.4565,125.667960,125.245040
2020-12-30 15:15:00-05:00,125.350,125.41,125.32,125.36,3735,125.354860,125.385565,-0.030705,-0.011130,-0.011130,-0.019575,0.203498,125.4590,125.662498,125.255502
2020-12-30 15:30:00-05:00,125.410,125.67,125.37,125.55,7877,125.384881,125.397745,-0.012864,-0.011477,-0.011477,-0.001387,0.211106,125.4520,125.663106,125.240894
2020-12-30 15:45:00-05:00,125.520,125.71,125.25,125.33,34033,125.376438,125.392727,-0.016289,-0.012439,-0.012439,-0.003850,0.228884,125.4390,125.667884,125.210116


In [31]:
drop_columns = ['close_12_ema','close_26_ema','macd', 'macd_9_ema', 'macds', 'macdh','kcmid']
df_final.drop(drop_columns, inplace=True, axis=1)
df_final.tail()

,open,high,low,close,volume,atr,kcup,kclo
time,,,,,,,,
2020-12-30 14:45:00-05:00,125.285,125.46,125.26,125.29,7778,0.219264,125.661264,125.222736
2020-12-30 15:00:00-05:00,125.325,125.36,125.25,125.31,5098,0.211460,125.667960,125.245040
2020-12-30 15:15:00-05:00,125.350,125.41,125.32,125.36,3735,0.203498,125.662498,125.255502
2020-12-30 15:30:00-05:00,125.410,125.67,125.37,125.55,7877,0.211106,125.663106,125.240894
2020-12-30 15:45:00-05:00,125.520,125.71,125.25,125.33,34033,0.228884,125.667884,125.210116


In [32]:
# Bollinger Band function to capture signal
def bollinger_band_signal_generator(dataframe_name, closing_price_column_name = 'close', bollinger_band_window = 20, num_standard_deviation = 2):
    """Creates Bollinger Band indicator with signal for long position based on closing price
    Args:
        dataframe_name (dict): Single security dataframe containing at least closing prices
        closing_price_column_name (str): Name of column in dataframe containing closing prices
        bollinger_band_window (int): Desired timeframe window used for rolling calculations
        num_standard_deviation (int): Desired number of standard deviations to calculate
    Returns:
        A dataframe of:
            original data passed to function,
            bollinger_band_middle (flt): Column of values for middle band,
            bollinger_band_std (flt): Column of values to calculate standard deviation,
            bollinger_band_upper (flt): Column of values for upper band,
            bollinger_band_lower (flt): Column of values for lower band,
            bollinger_band_long (flt): Column of generated signals (1.0 = True, 0.0 = False)
    """
    # Calculate mean and standard deviation
    dataframe_name['bollinger_band_middle'] = dataframe_name[closing_price_column_name].rolling(window=bollinger_band_window).mean()
    dataframe_name['bollinger_band_std'] = dataframe_name[closing_price_column_name].rolling(window=bollinger_band_window).std()

    # Calculate upper bollinger band and lower bollinger band
    dataframe_name['bollinger_band_upper'] = dataframe_name['bollinger_band_middle'] + (dataframe_name['bollinger_band_std'] * num_standard_deviation)
    dataframe_name['bollinger_band_lower'] = dataframe_name['bollinger_band_middle'] - (dataframe_name['bollinger_band_std'] * num_standard_deviation)

    # Create signal column
    dataframe_name['bollinger_band_long'] = np.where(dataframe_name[closing_price_column_name] > dataframe_name['bollinger_band_upper'], 1.0, 0.0)
    
    # Drop NaN values
    dataframe_name.dropna(inplace=True)

    # Return dataframe with features and target
    return dataframe_name


In [33]:
new_dataframe = bollinger_band_signal_generator(df_final)


In [34]:
new_dataframe.head()


,open,high,low,close,volume,atr,kcup,kclo,bollinger_band_middle,bollinger_band_std,bollinger_band_upper,bollinger_band_lower,bollinger_band_long
time,,,,,,,,,,,,,
2020-01-02 14:15:00-05:00,140.575,140.770,140.525,140.73,20048,0.229106,140.427356,139.969144,140.19825,0.420517,141.039285,139.357215,0.0
2020-01-02 14:30:00-05:00,140.745,140.870,140.690,140.87,13698,0.224568,140.482818,140.033682,140.25825,0.426742,141.111734,139.404766,0.0
2020-01-02 14:45:00-05:00,140.845,140.980,140.800,140.98,22353,0.220533,140.544283,140.103217,140.32375,0.432200,141.188150,139.459350,0.0
2020-01-02 15:00:00-05:00,140.960,140.985,140.865,140.87,70068,0.211604,140.596854,140.173646,140.38525,0.417030,141.219311,139.551189,0.0
2020-01-02 15:15:00-05:00,140.865,140.890,140.780,140.82,75963,0.202733,140.658983,140.253517,140.45625,0.357023,141.170296,139.742204,0.0


In [35]:
# Set the EMAs windows

fast_ema = 9
slow_ema = 21

In [36]:
# create EMAs columns
new_dataframe['EMA9'] = new_dataframe['close'].ewm(span=fast_ema, adjust=False).mean()
new_dataframe['EMA21'] = new_dataframe['close'].ewm(span=slow_ema, adjust=False).mean()

In [37]:
new_dataframe.head()

,open,high,low,close,volume,atr,kcup,kclo,bollinger_band_middle,bollinger_band_std,bollinger_band_upper,bollinger_band_lower,bollinger_band_long,EMA9,EMA21
time,,,,,,,,,,,,,,,
2020-01-02 14:15:00-05:00,140.575,140.770,140.525,140.73,20048,0.229106,140.427356,139.969144,140.19825,0.420517,141.039285,139.357215,0.0,140.730000,140.730000
2020-01-02 14:30:00-05:00,140.745,140.870,140.690,140.87,13698,0.224568,140.482818,140.033682,140.25825,0.426742,141.111734,139.404766,0.0,140.758000,140.742727
2020-01-02 14:45:00-05:00,140.845,140.980,140.800,140.98,22353,0.220533,140.544283,140.103217,140.32375,0.432200,141.188150,139.459350,0.0,140.802400,140.764298
2020-01-02 15:00:00-05:00,140.960,140.985,140.865,140.87,70068,0.211604,140.596854,140.173646,140.38525,0.417030,141.219311,139.551189,0.0,140.815920,140.773907
2020-01-02 15:15:00-05:00,140.865,140.890,140.780,140.82,75963,0.202733,140.658983,140.253517,140.45625,0.357023,141.170296,139.742204,0.0,140.816736,140.778097


In [38]:
updated_df = new_dataframe[['close', 'kcup', 'kclo', 'bollinger_band_upper', 'bollinger_band_lower', 'EMA9', 'EMA21']]

In [39]:
updated_df.head()

,close,kcup,kclo,bollinger_band_upper,bollinger_band_lower,EMA9,EMA21
time,,,,,,,
2020-01-02 14:15:00-05:00,140.73,140.427356,139.969144,141.039285,139.357215,140.730000,140.730000
2020-01-02 14:30:00-05:00,140.87,140.482818,140.033682,141.111734,139.404766,140.758000,140.742727
2020-01-02 14:45:00-05:00,140.98,140.544283,140.103217,141.188150,139.459350,140.802400,140.764298
2020-01-02 15:00:00-05:00,140.87,140.596854,140.173646,141.219311,139.551189,140.815920,140.773907
2020-01-02 15:15:00-05:00,140.82,140.658983,140.253517,141.170296,139.742204,140.816736,140.778097


In [40]:
updated_df.to_csv('../Resources/clean_indicators.csv')
